In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier, LogisticRegression

import torch 
import torch.nn as nn 
import torch.optim as optim

import copy
import tqdm

import util
import os 

import pickle

In [2]:
cwd = os.getcwd()
print(cwd)

/home/linhnm/msc_code/big_data_mining/eabdt/python


In [3]:
dataset = 'mix'

training_batch_from = 0
training_batch_to = 3000
val_batch_from = 3000
val_batch_to = 3600

testing_batch_from = 0
testing_batch_to = 400

In [4]:
X_train = util.load_text(os.path.join(cwd, '../data/raw/mix/training_set'), training_batch_from, training_batch_to)
y_train = util.load_label(os.path.join(cwd, '../data/raw/mix/training_set'), training_batch_from, training_batch_to)

# X_val = util.load_vector(os.path.join(cwd, '../data/vectorize/mix/training_set'), val_batch_from, val_batch_to)
# y_val = util.load_label(os.path.join(cwd, '../data/raw/mix/training_set'), val_batch_from, val_batch_to)

X_test = util.load_text(os.path.join(cwd, '../data/raw/mix/testing_set'), testing_batch_from, testing_batch_to)
y_test = util.load_label(os.path.join(cwd, '../data/raw/mix/testing_set'), testing_batch_from, testing_batch_to)

In [5]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(3000000, 1) (3000000, 1)
(400000, 1) (400000, 1)


In [6]:
y_test.shape

(400000, 1)

In [7]:
X_train = X_train['review_text']
X_test = X_test['review_text']

In [8]:
X_train.head()

0    stuning even for the non gamer  this sound tra...
1    the best soundtrack ever to anything   i m rea...
2    amazing   this soundtrack is my favorite music...
3    excellent soundtrack  i truly like this soundt...
4    remember  pull your jaw off the floor after he...
Name: review_text, dtype: object

In [9]:
# count_vect = CountVectorizer()

In [10]:
# X_train_counts = count_vect.fit_transform(X_train)
# X_test_counts = count_vect.fit_transform(X_test)

In [11]:
# tfidf_transformer = TfidfTransformer()

In [12]:
# X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
# X_test_tfidf = tfidf_transformer.fit_transform(X_test_counts)

In [13]:
# print(X_train_tfidf.shape, X_test_tfidf.shape)

In [14]:
# clf = MultinomialNB()

In [15]:
# clf.fit(X_train_tfidf, y_train)

In [16]:
text_clf = Pipeline([
      ('vect', CountVectorizer()),
      ('tfidf', TfidfTransformer()),
      ('clf', MultinomialNB()),
      ])

In [17]:
text_clf.fit(X_train, y_train)

/home/linhnm/msc_code/big_data_mining/eabdt/.venv/lib/python3.10/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [18]:
text_clf.score(X_test, y_test)

0.8480875

In [19]:
text_clf_sgd = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                           alpha=1e-3, random_state=42,
                          max_iter=5, tol=None)),
    ])

In [20]:
text_clf_sgd.fit(X_train, y_train)

/home/linhnm/msc_code/big_data_mining/eabdt/.venv/lib/python3.10/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 SGDClassifier(alpha=0.001, max_iter=5, random_state=42,
                               tol=None))])

In [21]:
text_clf_sgd.score(X_test, y_test)

0.8514425

In [22]:
text_clf_lg = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', LogisticRegression(random_state=42, max_iter=10000)),
    ])

In [23]:
text_clf_lg.fit(X_train, y_train)

/home/linhnm/msc_code/big_data_mining/eabdt/.venv/lib/python3.10/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(max_iter=10000, random_state=42))])

In [24]:
text_clf_lg.score(X_test, y_test)

0.911755

In [25]:
nb_path = '../model/sk_nb.sav'
sgd_path = '../model/sk_sgd.sav'
lg_path = '../model/sk_lg.sav'

In [26]:
pickle.dump(text_clf, open(nb_path, 'wb'))
pickle.dump(text_clf_sgd, open(sgd_path, 'wb'))
pickle.dump(text_clf_lg, open(lg_path, 'wb'))